In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)from keras.models import Sequential
from keras.models import Sequential
from keras.layers import Dropout, Flatten, GlobalAveragePooling2D,Conv2D
from keras.layers import Activation,Dropout,Flatten,Dense,MaxPooling2D
from keras.applications import ResNet50
from keras.optimizers import Adam
from keras.models import Model
from keras import regularizers
from keras.preprocessing.image import ImageDataGenerator
import math


In [ ]:
from keras.models import Sequential
from keras.models import Model
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard
from keras import optimizers, losses, activations, models
from keras.layers import Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, Concatenate
from keras import applications

In [ ]:
X_train = "../input/train/train"
Y_train = "../input/test/test"

In [ ]:
len(X_train)

In [ ]:
train_data = pd.read_csv('../input/train.csv',low_memory=False)
train_data['has_cactus'].value_counts()
train_data.has_cactus = train_data.has_cactus.astype(str)

In [ ]:
print("The number of rows in test set is %d"%(len(os.listdir('../input/test/test'))))
print("The number of rows in train set is %d"%(len(os.listdir('../input/train/train'))))

In [ ]:
#datagen = ImageDataGenerator(rescale=1./255) 
batch_size = 16

In [ ]:
train_datagen = ImageDataGenerator(rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,vertical_flip=False,
        fill_mode='nearest')

In [ ]:
valid_datagen = ImageDataGenerator(rescale=1/.255)

In [ ]:
train_generator=train_datagen.flow_from_dataframe(train_data[:15001],directory=X_train,x_col='id',y_col='has_cactus',
                                           class_mode='binary',batch_size=batch_size,target_size=(299,299))

valid_generator = valid_datagen.flow_from_dataframe(train_data[15000:],directory=X_train,x_col='id',y_col='has_cactus',
                                           class_mode='binary',batch_size=batch_size,target_size=(299,299))

In [ ]:
train_generator.class_indices

In [ ]:
from keras.applications import InceptionV3

In [ ]:
base_model = InceptionV3(weights='imagenet',include_top=False,input_shape=(299,299,3))
print('Model Loaded')

base_model.trainable = False


In [ ]:
top_model = Sequential()
top_model.add(base_model)
top_model.add(Flatten())
top_model.add(Dense(256))
top_model.add(Activation('relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1))
top_model.add(Activation('sigmoid'))

model=top_model

model.compile(optimizer=Adam(lr=0.005),
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()




In [ ]:
file_path="weights.best.hdf5"

checkpoint = ModelCheckpoint(file_path,monitor='acc', verbose=1, save_best_only=True, mode='max')

#early = EarlyStopping(monitor="acc", mode="max", patience=15)

#callbacks_list = [checkpoint, early] 

history = model.fit_generator(train_generator,
                    steps_per_epoch= train_generator.n// train_generator.batch_size,
                    epochs= 10 , verbose=2,
                    validation_data=valid_generator,
                    validation_steps=valid_generator.n // valid_generator.batch_size,
                    callbacks= [checkpoint])